# Import Libraries

In [2]:
import pandas as pd
import dash
import numpy as np
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import pandas as pd
from collections import Counter
from plotly import graph_objs as go
from IPython.display import Video

<b> *This Dashboard uses the file 'Primary_Dashboard.csv' which is the Consumption+Avg Price data for all US* </b>        
<b> *This Dashboard uses the file 'acv_Consumption_df.csv' which is the Consumption+ACV data for all US* </b>

# Reading the csv files

In [2]:
avgprice_consumption_us=pd.read_csv('Primary_Dashboard.csv',parse_dates=['Cleaned Date'])

In [3]:
acv_consumption_us=pd.read_csv('acv_Consumption_df.csv',parse_dates=['Cleaned Date'])


# Preparing the datasets for Dash Chart input

In [4]:
avgprice_consumption_us.Product_SKU.unique()

avgprice_consumption_us['Size']=avgprice_consumption_us.Size.str.strip()



In [5]:
acv_consumption_us.Product_SKU.unique()

acv_consumption_us['Size']=acv_consumption_us.Size.str.strip()

In [6]:
avgprice_consumption_us['Size']=avgprice_consumption_us['Size'].fillna ('NA')

In [7]:
acv_consumption_us['Size']=acv_consumption_us['Size'].fillna ('NA')

# Dash Chart Codes

In [9]:


external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

df=avgprice_consumption_us
acv_df=acv_consumption_us
years = df['Consumption_Year'].unique()
categories=df['Category'].unique()
sizes=df['Size'].unique()
fig = go.Figure()
fig1=go.Figure()
all_options={}

for c in categories:
    all_options[c]=df[df['Category']==c]['Size'].unique()

app.layout = html.Div([
    html.Div([

        html.Div([ html.Label(["Year",
            dcc.Dropdown(
                id='year',
                options=[{'label': y, 'value': y} for y in years],
                value=2020
            )])
        ],
        style={'width': '20%', 'display': 'inline-block'}),

        html.Div([ html.Label(["Category",
            dcc.Dropdown(
                id='category',
                options=[{'label': c, 'value': c} for c in categories],
                value='Dinner'
            )])

        ],
        style={'width': '20%', 'display': 'inline-block'}),
        html.Div([ html.Label(["Size",
            dcc.Dropdown(id='size')])],
            style={'width': '20%', 'display': 'inline-block'})
    ]),

    html.Div([dcc.Graph(
        id='indicator-graphic',
        figure=fig
    )],
        style={'height': '30%','width': '90%'}),
    html.Div([dcc.Graph(
        id='second_graph',
        figure=fig
    )],
        style={'height': '30%','width': '90%'}),
    html.Div([dcc.Graph(
        id='third_graph',
        figure=fig1
    )],
        style={'height': '30%','width': '90%'})
])

@app.callback(
    Output('size', 'options'),
    [Input('category', 'value')])
def set_cities_options(selected_category):
    return [{'label': i, 'value': i} for i in all_options[selected_category]]



@app.callback(
    Output('size', 'value'),
    [Input('size', 'options')])
def set_cities_value(available_options):
    return available_options[0]['value']


@app.callback(
    [Output('indicator-graphic', 'figure'),Output('second_graph', 'figure'),Output('third_graph', 'figure')],
    [Input('year', 'value'),
     Input('category', 'value'),Input('size', 'value')])
def update_graph(year_value,category_value,size):
    fig = go.Figure()
    dff = df.loc[(df['Consumption_Year'] == year_value) & (df['Category'] == category_value) & (df['Size'] == size)]
    acv_dff=acv_df.loc[(acv_df['Consumption_Year'] == year_value) & (acv_df['Category'] == category_value) & (acv_df['Size'] == size)]
    dff1=dff.groupby(['Consumption_Year','Category','Size','Cleaned Date']).agg({'Consumption':'sum','Avg Price':'mean'}).reset_index()
    dff2=acv_dff.groupby(['Consumption_Year','Category','Size','Cleaned Date']).agg({'ACV':'max'}).reset_index()
    tickdates=dff.groupby(['Consumption_Month']).agg({'Cleaned Date':'min'}).reset_index()

    fig = go.Figure(data=[go.Scatter(x=dff1['Cleaned Date'],y=dff1['Consumption']
                                     ,mode='lines+markers',line=dict(color='blue'))])
    
    fig1 = go.Figure(data=[go.Scatter(x=dff1['Cleaned Date'],y=dff1['Avg Price'],mode='lines+markers',line=dict(color='orange'))])
    
    fig2 = go.Figure(data=[go.Scatter(x=dff2['Cleaned Date'],y=dff2['ACV'],mode='lines+markers',line=dict(color='pink'))])
    
    fig.update_layout(
                  yaxis=dict(title=f'<b>Weekly Consumption(lb)</b>'),height=250,
    xaxis=dict(title=f'<b>Weekend Date</b>',showgrid=True,showticklabels = True,tickmode='array',tickvals = np.array(tickdates['Cleaned Date']),
               ticktext=np.array(['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec'])) ,   
    showlegend=False, margin={'l':30, 'b': 0, 't': 10, 'r': 10},
    )

    fig1.update_layout(yaxis=dict(title=f'<b>Weekly Avg Price($)</b>'),height=150,
    xaxis=dict(title=f'<b>Weekend Date</b>',showgrid=True), hovermode='closest',margin={'l': 10, 'b': 0, 't': 15, 'r': 10})
    
    fig2.update_layout(yaxis=dict(title=f'<b>Max of ACV(%)</b>'),height=150,
    xaxis=dict(title=f'<b>Weekend Date</b>',showgrid=True), hovermode='closest',margin={'l': 10, 'b': 0, 't': 20, 'r': 10})
    
    
    return fig,fig1,fig2
# fig.show()

if __name__ == "__main__":
    app.run_server(mode='inline', port = 8050,debug=False)

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Nov/2020 20:07:37] "GET /_alive_b9e452b8-314a-45d2-b4fc-822b94c24c6f HTTP/1.1" 200 -


127.0.0.1 - - [13/Nov/2020 20:07:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:38] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:38] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:38] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:47] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:47] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:47] "GET /_favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [13/Nov/2020 20:07:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 

# Demo Video of the chart

In [3]:
Video("Dash.mp4",embed=True)